In [83]:
import xmltodict
import pandas as pd
from shapely.geometry import LineString, Point
from shapely.ops import nearest_points
import glob
from pathlib import Path

with open("../data/roads.osm", "r", encoding="utf-8") as file:
    osm_data = xmltodict.parse(file.read()) # XML 문서를 파이썬 딕셔너리로 변환

# GPS 로그 Pandas DataFrame 객체 생성 - 행,열 구
df = pd.read_csv("../gps_logs/gps_straight01.csv")  # 경로가 맞는지 확인

# 모든 노드를 딕셔너리로 정리
nodes_raw = osm_data["osm"]["node"]          # 노드가 하나면 dict, 여러 개면 list
if isinstance(nodes_raw, dict):              # 노드가 1개뿐인 edge-case 대비 - 이경우 dict로 반환됨
    nodes_raw = [nodes_raw]

nodes = {}
for n in nodes_raw:
    nid = int(n["@id"])
    lat = float(n["@lat"])
    lon = float(n["@lon"])
    nodes[nid] = (lon, lat)                  # (경도, 위도) 순서로 저장

print("총 노드 개수:", len(nodes))
print("예시 3개:", list(nodes.items())[:3])

총 노드 개수: 113
예시 3개: [(436854331, (127.02421, 37.4969787)), (1906756396, (127.0259134, 37.4975028)), (2280678660, (127.0294169, 37.4984367))]


In [84]:
ways_raw = osm_data["osm"]["way"]
if isinstance(ways_raw, dict):                  # way가 1개일 가능성 대비
    ways_raw = [ways_raw]

print("총 way 개수:", len(ways_raw))

총 way 개수: 31


In [85]:
all_roads = {}
missing_report = []          # (way_id, missing_nids) 목록 기록용

for w in ways_raw:
    wid = int(w["@id"])
    node_refs = [int(nd["@ref"]) for nd in w["nd"]]

    # 존재하는 노드만 추출
    coords = []
    missing = []
    for nid in node_refs:
        if nid in nodes: # 정의되지 않은 노드 필터링.
            coords.append(nodes[nid])
        else:
            missing.append(nid)

    # 누락 노드 기록
    if missing:
        missing_report.append((wid, missing))

    # 좌표가 2개 이상이면 LineString 생성
    if len(coords) >= 2:
        all_roads[wid] = LineString(coords)

print(f"선 생성 완료: {len(all_roads)}개")
if missing_report:
    print(f"⚠️  누락 노드가 있는 way: {len(missing_report)}개 (아래 일부 예시)")
    for wid, miss in missing_report[:5]:
        print(f"  way {wid} → missing {len(miss)} nodes (ex: {miss[:3]} …)")
# 누락노드 제외한 way의 노드 개수 확인
for wid, line in all_roads.items():
    way_tag = next(w for w in ways_raw if int(w["@id"]) == wid)
    original_n   = len(way_tag["nd"])   # XML에 기록된 nd 개수
    actual_n     = len(line.coords)     # 존재하는 노드만으로 만든 개수
    if original_n != actual_n:
        print(f"way {wid}: XML {original_n}개 → 실제 {actual_n}개")
print(f"✅ 생성된 LineString 수: {len(all_roads)}")   

# 전부 출력해보기
for i, (wid, line) in enumerate(all_roads.items()):
    
    print(f"way {wid}: {len(line.coords)} pts, "
          f"첫점 {line.coords[0]} → 끝점 {line.coords[-1]}")

선 생성 완료: 31개
⚠️  누락 노드가 있는 way: 3개 (아래 일부 예시)
  way 472042763 → missing 1 nodes (ex: [11034387027] …)
  way 472042765 → missing 1 nodes (ex: [11034387028] …)
  way 504231011 → missing 1 nodes (ex: [11245176923] …)
way 472042763: XML 3개 → 실제 2개
way 472042765: XML 3개 → 실제 2개
way 504231011: XML 4개 → 실제 3개
✅ 생성된 LineString 수: 31
way 218864485: 4 pts, 첫점 (127.0276806, 37.4980478) → 끝점 (127.0274875, 37.4984961)
way 218864491: 15 pts, 첫점 (127.0294169, 37.4984367) → 끝점 (127.0369935, 37.5007121)
way 218873820: 5 pts, 첫점 (127.0280994, 37.4966908) → 끝점 (127.0289478, 37.4949431)
way 218877837: 5 pts, 첫점 (127.0260403, 37.5009804) → 끝점 (127.0267729, 37.4994695)
way 239954010: 10 pts, 첫점 (127.0284946, 37.4982952) → 끝점 (127.0274875, 37.4984961)
way 304445274: 2 pts, 첫점 (127.0274725, 37.4979843) → 끝점 (127.0269036, 37.497806)
way 305539406: 9 pts, 첫점 (127.0272447, 37.4984526) → 끝점 (127.0269036, 37.497806)
way 375049565: 14 pts, 첫점 (127.0369107, 37.5009049) → 끝점 (127.0284946, 37.4982952)
way 472042763: 2

In [86]:

DEG2M = 111_320    
# 위도 1° ≈ 111.32 km (서울 근사) 
# Shapely가 계산한 거리는 ‘도(degree)’ 단위이므로, m 단위로 바꿀 때 곱해 주기 위함

def find_closest_road(lon, lat, roads_dict):
    """하나의 GPS 점(lon, lat)을 roads_dict 중 가장 가까운 선으로 매칭."""
    p = Point(lon, lat)
    best_way, best_dist, best_proj = None, float("inf"), None

    for wid, line in roads_dict.items():
        d = p.distance(line)           # 단위: degree
        if d < best_dist:
            best_way, best_dist = wid, d
            best_proj = nearest_points(p, line)[1]   # 수직으로 내려서 구한 도로 위 점(투영점)
            # p와 line 사이에서 가장 가까운 두 점을 반환
            
    return best_way, best_dist * DEG2M, best_proj

# GPS의 첫 행만 테스트
test = df.iloc[0]
wid, dist_m, proj = find_closest_road(test.Longitude, test.Latitude, all_roads)

print("가장 가까운 way:", wid)
print(f"거리: {dist_m:.1f} m")
print("투영점:", (proj.x, proj.y))


가장 가까운 way: 218873820
거리: 2.3 m
투영점: (127.02813114879656, 37.496624035596895)
